In [5]:
from __future__ import division # For python 2.*

import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn import *
from sklearn.metrics import roc_curve, auc

np.random.seed(0)
%matplotlib inline

X = np.genfromtxt('uci-cs178-f21/X_train.txt', delimiter=',')
Y = np.genfromtxt('uci-cs178-f21/Y_train.txt', delimiter=',')

X,Y = ml.shuffleData(X,Y)
# Fetureset1_X = X

# X = X[:,0:41]
Xtr, Xva, Ytr, Yva = ml.splitData(X, Y, 0.75)

Xte = np.genfromtxt('uci-cs178-f21/X_test.txt', delimiter=',')
# Xte = Xte[:,:41]

Xtr, param = ml.transforms.rescale(Xtr)
Xva, _ = ml.transforms.rescale(Xva,param)
Xte, _ = ml.transforms.rescale(Xte,param)

In [ ]:
print(Xte.shape)
print(Xtr.shape)
print(Xva.shape)

In [ ]:
# print(X.shape)
# print(Y.shape)
# print(Xte.shape)

# Fetureset1_X = X[:,0:41]
# print(Fetureset1_X.shape)
# Fetureset2_X = X[:,41:69]
# print(Fetureset2_X.shape)
# Fetureset3_X = X[:,69:107]
# print(Fetureset3_X.shape)

# print(Fetureset1_X[0])
# print(Fetureset2_X[0])
# print(Fetureset3_X[0])

# # for i in X[:1]:
# #     for j in i :
# #         x += 1
# #         print(x,j)


In [ ]:
# nBag = 100
# classifiers2 = [ None ] * nBag

# mt,nt = Xte.shape

# Ye = np.zeros((mt,2))
# auc_valist = []
# auc_trlist = []
# for i in range(nBag):
#     Xi,Yi = ml.bootstrapData(Xtr,Ytr,mt) 
#     classifiers2[i] = ml.dtree.treeClassify(Xi, Yi, maxDepth = 25 , minLeaf = 4, minParent = 512, nFeatures = 41)
#     #Train a model on data Xi,Yi
#     auc_valist.append(classifiers2[i].auc(Xva,Yva))
#     auc_valist.append(classifiers2[i].auc(Xtr,Ytr))
#     #Train a model on data Xi,Yi
#     Ye += classifiers2[i].predictSoft(Xte)
# learner = Ye / 25

# print("train:",np.mean(auc_trlist))
# print("valadation:",np.mean(auc_valist))

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn import tree
from sklearn import gaussian_process
from sklearn import linear_model


In [7]:
rf = RandomForestClassifier(n_estimators= 100, min_samples_split = 512,max_features = 41,min_samples_leaf = 4, bootstrap=True, max_depth=25)
rf.fit(Xtr, Ytr)

pred_soft = rf.predict_proba(Xtr)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
roc_auc_tr = auc(false_positive_rate, true_positive_rate)

pred_soft = rf.predict_proba(Xva)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
roc_auc_va = auc(false_positive_rate, true_positive_rate)

print ("max_features = " + str(41))
print ("Training: " + str(roc_auc_tr))
print ("Validation: " + str(roc_auc_va))


max_features = 41
Training: 0.7720928791450534
Validation: 0.7261921212297152


In [ ]:
pred_soft[:,1]

In [34]:
max_va = 0
best_number = 0

for mf in [30,40,50,60,70,80,90,100,107]:
    rf = RandomForestClassifier(n_estimators= 100, min_samples_split = 64,max_features = mf, min_samples_leaf = 4, bootstrap=True, max_depth=25)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max_va:
        max_va = roc_auc_va
        best_number = mf
    
    print ("maxF = " + str(mf))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
    
print ("max", best_number)
print ("max_va",max_va)

maxF = 30
Training: 0.9333753766469832
Validation: 0.7457926669956745
maxF = 40
Training: 0.9351316424777755
Validation: 0.7432879496037391
maxF = 50
Training: 0.9373484052372972
Validation: 0.7462107004964147
maxF = 60
Training: 0.9381161583169594
Validation: 0.7490033965221936
maxF = 70
Training: 0.9383335614086274
Validation: 0.7456091967370163
maxF = 80
Training: 0.9428591346737274
Validation: 0.7460307138502627
maxF = 90
Training: 0.9410150813712403
Validation: 0.7506952709960228
maxF = 100
Training: 0.9396631543071373
Validation: 0.7488605684094406
maxF = 107
Training: 0.9393152319009345
Validation: 0.7451853572154323
max 90
max_va 0.7506952709960228


In [36]:
max_va = 0
best_number_for_bag = 0

for bag in [100,200,300,400,500,600]:
    rf = RandomForestClassifier(n_estimators= bag, min_samples_split = 64,max_features = 90, min_samples_leaf = 4, bootstrap=True, max_depth=25)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max_va:
        max_va = roc_auc_va
        best_number_for_bag = bag
    
    print ("bag = " + str(bag))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
    
print ("max", best_number_for_bag)

bag = 100
Training: 0.9405640086857959
Validation: 0.7481801027665689
bag = 200
Training: 0.9425206365108095
Validation: 0.7478120010450838
bag = 300
Training: 0.9432393318857009
Validation: 0.7497245043109705
bag = 400
Training: 0.9436458653391825
Validation: 0.7483879583127704
bag = 500
Training: 0.9437767719513864
Validation: 0.7499172642029786
bag = 600
Training: 0.9440511078004332
Validation: 0.750799779371208
max 600


In [37]:
max_va = 0
best_number_for_parents = 0

for parents in [2**5,2**6,2**7,2**8,2**9,2**10]:
    rf = RandomForestClassifier(n_estimators= 600, min_samples_split = parents,max_features = 90, min_samples_leaf = 4, bootstrap=True, max_depth=25)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max_va:
        max_va = roc_auc_va
        best_number_for_parents = parents
    
    print ("parents = " + str(parents))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
    
print ("max", best_number_for_parents)

parents = 32
Training: 0.9775712046764902
Validation: 0.7487688332801115
parents = 64
Training: 0.9435409076708176
Validation: 0.7507080442418788
parents = 128
Training: 0.8836579748463074
Validation: 0.7488025082010045


KeyboardInterrupt: 

In [22]:
max_va = 0
best_number_for_leaf = 0

for leaf in [1,2,3,4,5,6,7,8,9,10,15,20,25,30]:
    rf = RandomForestClassifier(n_estimators= 100, min_samples_split = 64 ,max_features = 90, min_samples_leaf = leaf, bootstrap=True, max_depth=25)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max_va:
        max_va = roc_auc_va
        best_number_for_leaf = leaf
    
    print ("leaf = " + str(leaf))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
    
print ("leaf", best_number_for_leaf)

leaf = 1
Training: 0.9474442226806679
Validation: 0.7397544053183149
leaf = 2
Training: 0.9423189835243572
Validation: 0.7415124684297616
leaf = 3
Training: 0.9393255598387809
Validation: 0.7459633640084768
leaf = 4
Training: 0.9342771347202032
Validation: 0.7506871425668418
leaf = 5
Training: 0.9299281691922778
Validation: 0.7457357679914071
leaf = 6
Training: 0.9243327506655065
Validation: 0.7377373936772433
leaf = 7
Training: 0.9252760786885585
Validation: 0.7464673266177027
leaf = 8
Training: 0.9204640342474419
Validation: 0.7447255203646181
leaf = 9
Training: 0.9191059104206311
Validation: 0.744071762417627
leaf = 10
Training: 0.9207825220307824
Validation: 0.7461572851046536
leaf = 15
Training: 0.9076546802341343
Validation: 0.7476354980114378
leaf = 20
Training: 0.8959483499828297
Validation: 0.745639388045403
leaf = 25
Training: 0.8938074975664796
Validation: 0.7358039887363197
leaf = 30
Training: 0.8851639172835459
Validation: 0.7404882863529479
leaf 4


In [23]:
max_va = 0
best_number_for_depth = 0

for depth in [10,15,20,25,30,35]:
    rf = RandomForestClassifier(n_estimators= 100, min_samples_split = 64 ,max_features = 36, min_samples_leaf = 4, bootstrap=True, max_depth=depth)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max_va:
        max_va = roc_auc_va
        best_number_for_depth = depth
    
    print ("depth = " + str(depth))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
    
print ("depth", best_number_for_depth)

depth = 10
Training: 0.8720657683082063
Validation: 0.7439695764507794
depth = 15
Training: 0.9227040348671182
Validation: 0.7456324208203906
depth = 20
Training: 0.9332349166922713
Validation: 0.7451656167445642
depth = 25
Training: 0.934066057490466
Validation: 0.7488071530176794
depth = 30
Training: 0.9325453977417965
Validation: 0.748230034545824
depth = 35
Training: 0.9337780371237726
Validation: 0.7423020872644932
depth 25


In [39]:
rf = RandomForestClassifier(n_estimators= 500, min_samples_split = 64 ,max_features = 90, min_samples_leaf = 4, bootstrap=True, max_depth=25)
rf.fit(Xtr, Ytr)

pred_soft = rf.predict_proba(Xtr)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
roc_auc_tr = auc(false_positive_rate, true_positive_rate)

pred_soft = rf.predict_proba(Xva)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
roc_auc_va = auc(false_positive_rate, true_positive_rate)

print ("Training: " + str(roc_auc_tr))
print ("Validation: " + str(roc_auc_va))

Training: 0.9445910007513574
Validation: 0.7485435596713793


In [41]:
max_va = 0
best_number_for_features = 0

total = 0
for mf in range(90,99):
    rf = RandomForestClassifier(n_estimators= 500, min_samples_split = 64 ,max_features = mf, min_samples_leaf = 4, bootstrap=True, max_depth=25)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max_va:
        max_va = roc_auc_va
        best_number_for_features = mf
    
    
    print ("depth = " + str(mf))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
total /=9   
print ("mean",total)
print ("mf", best_number_for_depth)

depth = 90
Training: 0.9440030828894472
Validation: 0.7480535315121781
depth = 91
Training: 0.9428477739420965
Validation: 0.7471919180189857
depth = 92
Training: 0.9445256765444786
Validation: 0.7485214967921735
depth = 93
Training: 0.9443522962878809
Validation: 0.7496641216941968
depth = 94
Training: 0.94414819041619
Validation: 0.7489894620721689
depth = 95
Training: 0.9442049940743457
Validation: 0.7487607048509304
depth = 96
Training: 0.9445524000836562
Validation: 0.7472348825732283
depth = 97
Training: 0.9453359032685341
Validation: 0.7475739541904955
depth = 98
Training: 0.9442761277462632
Validation: 0.7475809214155078
mean 0.0
mf 25
